## 0. 라이브러리

In [1]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. 데이터 불러오기

In [2]:
train_data_path ="~/aiffel/dktc/data2/train0.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data

,class,conversation
idx,,
1,일반 대화,야 여행 가기 너무 좋은 날씨다\n날씨 진짜 너무 좋아 **이는 일하기 싫을 날씨야...
2,갈취 대화,사정이 있었어요 . 살려주세요 . 뒤진 사람들 중에 사정 없는 사람 없어 맞아 . ...
3,갈취 대화,배고프다 그러게 야 만원만 빌려줘봐 오늘도 ? 저번에도 만원 빌려가고 안 돌려 줬잖...
4,갈취 대화,저기 실례지만 제가 휠체어를 타는 장애인이라서 아 혼자 휠체어 운전하시는게 어렵다고...
5,직장 내 괴롭힘 대화,김 사원 낙하산이라는 소문이 있던데 ? 요즘 시대에도 낙하산이 있어요 ? 요즘 더 ...
...,...,...
4826,일반 대화,티비는 잘 안 보니?\n티비 매일 보지\n너는?\n키키 요즘 핫한 스우파 안 보니?...
4827,갈취 대화,이거 나이키 신상이네 ? 응 아빠가 사주셨어 우와 멋지네 근데 너랑은 별로 안어울린...
4828,직장 내 괴롭힘 대화,일처리를 어떻게 한거야 ! 기간 내에 상품이 못오면 그사이 손실은 어떻게 할거냐고 ...


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [3]:
def preprocess_sentence(sentence):
    # synolp
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r'[^\w\s]', '', sentence)
    # base preprocess
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 엔터 구분 (\n)
    sentence = sentence.replace("\n", " ")
    sentence = sentence.strip()
    return sentence

### 2.1-2 전처리 함수 적용

In [4]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))

100%|██████████| 4830/4830 [00:01<00:00, 3323.84it/s]


### 2.2 최대 길이 지정

In [5]:
MAX_LEN = 300

### 2.3 class(label) 인코딩

In [6]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화','일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

len(labels)

4830

In [7]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)

Class mapping: {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}


### 2.4 train-val

In [8]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42, stratify=labels)

In [9]:
# train data 증강

def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return "".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = (
        new_words[random_idx_2],
        new_words[random_idx_1],
    )
    return new_words


def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


print("before data augmentation: ", len(train_sentences))

train_splted = pd.DataFrame({ "sentence": train_sentences, "class": train_labels })

# random deletion
train_splted_rd = train_splted.copy()
train_splted_rd["sentence"] = train_splted_rd["sentence"].apply(random_deletion)

# random swap
train_splted_rs = train_splted.copy()

# with data augmentation
train_concated = pd.concat([train_splted , train_splted_rd , train_splted_rs])

print("after data augmentation: ", len(train_concated))

train_concated

before data augmentation:  3864
after data augmentation:  11592


,sentence,class
idx,,
346,김비서 오늘 스케줄 어떻게 되지 네 오전에 바이어 미팅있고 점심후엔 임원회의 예정 ...,3
1779,이거 빨리 끝내고 일해야겠다 ㅎ 넌 몇 시 출근 난 새벽 시 반 울고 싶다 오후에는...,2
3214,김 과장 내가 부탁한 자료는 어떻게 된거야 아 네 부장님 아직 하고 있습니다 내가 ...,3
4225,학생 이것 좀 도와줄 수 있어 네 어떤 아니 우리가 밖에서 뭘 하는데 밖이 너무 추...,0
295,내가 지난번에 말한대로 좋게 말할 때 시키는대로 해 왜그래 나는 못하겠어 야 그건 ...,4
...,...,...
2779,야 술 받으라고 죄송합니다 한 잔 받겠습니다 너는 어째 회사 안이나 밖이나 똑같냐 ...,3
2786,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...,1
2932,지민이 포기해 애초에 너희 어울리지 않아 그건 지민이랑 제가 판단하는겁니다 글쎄 이...,4


In [10]:
train_sentences, train_labels = train_concated["sentence"], train_concated["class"].values
print(train_sentences[:5])

idx
346     김비서 오늘 스케줄 어떻게 되지 네 오전에 바이어 미팅있고 점심후엔 임원회의 예정 ...
1779    이거 빨리 끝내고 일해야겠다 ㅎ 넌 몇 시 출근 난 새벽 시 반 울고 싶다 오후에는...
3214    김 과장 내가 부탁한 자료는 어떻게 된거야 아 네 부장님 아직 하고 있습니다 내가 ...
4225    학생 이것 좀 도와줄 수 있어 네 어떤 아니 우리가 밖에서 뭘 하는데 밖이 너무 추...
295     내가 지난번에 말한대로 좋게 말할 때 시키는대로 해 왜그래 나는 못하겠어 야 그건 ...
Name: sentence, dtype: object


## 3. 모델
### 3.1-1 토크나이저 정의

In [11]:
# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

### 3.1-2 토크나이저 적용

In [12]:
# 데이터셋을 BERT 입력 형식으로 변환
train_encodings = tokenizer(train_sentences.values.reshape(-1).tolist(), truncation=True, padding=True, max_length=MAX_LEN) # 뒤쪽에 패딩
val_encodings = tokenizer(val_sentences, truncation=True, padding=True, max_length=MAX_LEN)

### 3.2 모델 준비

In [13]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3.3 파라미터

In [14]:
BATCH_SIZE = 16
lr = 5e-5
EPOCH = 10

### 3.4 TF 데이터셋 생성

In [15]:
# TensorFlow 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(100).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [18]:
early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=2,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

import datetime

now = datetime.datetime.now()
time = now.strftime("%y-%m-%d %H:%M")
data_type = 0

checkpoint = ModelCheckpoint(
    filepath=f'./models/klue_no_aug_weights_{data_type}_{time}.keras',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,   # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)

### 3.6-2 모델 훈련

In [19]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/10
725/725 [==============================] - 795s 1s/step - loss: 0.6929 - accuracy: 0.7382 - val_loss: 0.4128 - val_accuracy: 0.8551

Epoch 00001: val_loss improved from inf to 0.41277, saving model to ./models/klue_no_aug_weights_0_24-06-26 08:09.keras
Epoch 2/10
725/725 [==============================] - 779s 1s/step - loss: 0.3107 - accuracy: 0.8947 - val_loss: 0.3825 - val_accuracy: 0.8861

Epoch 00002: val_loss improved from 0.41277 to 0.38247, saving model to ./models/klue_no_aug_weights_0_24-06-26 08:09.keras
Epoch 3/10
725/725 [==============================] - 779s 1s/step - loss: 0.1866 - accuracy: 0.9375 - val_loss: 0.4127 - val_accuracy: 0.8882

Epoch 00003: val_loss did not improve from 0.38247
Epoch 4/10
725/725 [==============================] - 779s 1s/step - loss: 0.1335 - accuracy: 0.9574 - val_loss: 0.4411 - val_accuracy: 0.8892

Epoch 00004: val_loss did not improve from 0.38247


### 3.7 모델 평가

In [20]:
# 모델 평가
evaluation = model.evaluate(val_dataset)
print("평가 결과:", evaluation)

61/61 [==============================] - 21s 344ms/step - loss: 0.3825 - accuracy: 0.8861
평가 결과: [0.38247063755989075, 0.8861283659934998]


In [21]:
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np 

def score(model, val):
    X, y = [], []
    for batch in val_dataset:
        inputs, labels = batch
        X.append(inputs)
        y.append(labels)
    # 각 입력 키에 대해 데이터를 결합하여 numpy 배열로 변환
    X = {key: np.concatenate([d[key].numpy() for d in X], axis=0) for key in X[0].keys()}
    y = np.concatenate(y, axis=0)
    
    # 실제 예측값 생성
    real_predictions = model.predict(X)
    logits = real_predictions.logits

    # 예측값을 레이블로 변환
    if logits.ndim > 1:
        real_predicted_labels = np.argmax(logits, axis=1)
    else:
        real_predicted_labels = (logits > 0.5).astype(int)
    

    # 정확도 계산
    real_accuracy = accuracy_score(y, real_predicted_labels)
    print(f"Real Accuracy: {real_accuracy:.4f}")

    # 분류 보고서 생성
    real_report = classification_report(y, real_predicted_labels, target_names=[f"Class {i}" for i in range(logits.shape[1])])
    print(real_report)

    # F1 스코어 계산
    real_f1 = f1_score(y, real_predicted_labels, average='weighted')
    print(f"\nWeighted F1 Score (based on real predictions): {real_f1:.4f}")

In [22]:
score(model, val_dataset)

Real Accuracy: 0.8861
              precision    recall  f1-score   support

     Class 0       0.85      0.84      0.85       196
     Class 1       0.79      0.87      0.83       219
     Class 2       1.00      0.99      0.99       176
     Class 3       0.96      0.92      0.94       196
     Class 4       0.86      0.82      0.84       179

    accuracy                           0.89       966
   macro avg       0.89      0.89      0.89       966
weighted avg       0.89      0.89      0.89       966


Weighted F1 Score (based on real predictions): 0.8869


## 4. 모델 적용

In [23]:
test_data_path ="/aiffel/aiffel/dktc/data/test.json"

with open(test_data_path, "r", encoding="utf-8") as json_file:
    test = json.load(json_file)

In [24]:
import numpy as np

test_predict = list()

for key in test:
    test_sentence = test[key]['text']
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding=True, max_length=300, return_tensors="tf")
    
    test_predictions = model.predict({
        "input_ids": test_encodings["input_ids"],
        "token_type_ids": test_encodings["token_type_ids"],
        "attention_mask": test_encodings["attention_mask"]
    }) 
    test_class_probabilities = tf.nn.softmax(test_predictions.logits, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predict.append(test_predicted_class[0])

In [25]:
# {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}
#   협박 대화 : 0,  갈취 대화 : 1,  직장 내 괴롭힘 대화 : 2,  기타 괴롭힘 대화 : 3,  일반 대화 : 4
def labelnum_to_text(x):
    if x == 0:
        return '01'
    if x == 1:
        return '03'
    if x == 2:
        return '04'
    if x == 3:
        return '02'
    if x == 4:
        return '00'

import datetime
    
submission = pd.read_csv("../data/new_submission.csv")
submission["class"] = [ labelnum_to_text(pred) for pred in test_predict ]

now = datetime.datetime.now()
filename = now.strftime("../sub/submission %y-%m-%d %H:%M.csv")

submission.to_csv(filename, index=False)
submit_file = pd.read_csv(filename)

print(submit_file.shape)
print(submit_file.head())

(500, 2)
  file_name  class
0     t_000      1
1     t_001      2
2     t_002      2
3     t_003      3
4     t_004      3


In [26]:
# submission 0.658